<a href="https://colab.research.google.com/github/RaminParker/Gradio-Templates/blob/main/Switch_Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This notebook was inspired by [Sagnik220](https://github.com/Sagnik220/Iris-Classification-GRADIO/blob/main/Iris_Classification.ipynb)

In [ ]:
!pip install gradio
!pip install pycaret

Load the iris dataset from pycaret repo

In [ ]:
import gradio as gr
from pycaret.datasets import get_data
from pycaret.classification import *

In [ ]:
data = get_data('iris')

Initialize setup

In [ ]:
s = setup(data, target = 'species', session_id = 123)

In [ ]:
best = compare_models(n_select = 15)
compare_model_results = pull()

Creating a predict function to be passed into gradio UI

In [ ]:
def predict(model, sepal_length, sepal_width, petal_length, petal_width):
  
    # build dataframe
    df = pd.DataFrame.from_dict({'sepal_length': [sepal_length], 'sepal_width': [sepal_width], 
                                 'petal_length': [petal_length], 'petal_width': [petal_width]})
    
    modeles_list = list(compare_model_results['Model'])   # list of all models
    model_index = modeles_list.index(model)               # pick list index of requested model
    model = best[model_index]                             # pick requested model

    # make prediction
    pred = predict_model(model, df, raw_score=True)

    # return result
    return {'Iris-setosa': pred['Score_Iris-setosa'][0].astype('float64'), 
            'Iris-versicolor': pred['Score_Iris-versicolor'][0].astype('float64'),
            'Iris-virginica' : pred['Score_Iris-virginica'][0].astype('float64')}

Define Gradio interface

In [ ]:
model = gr.inputs.Dropdown(list(compare_model_results['Model']), label="Model")

sepal_length = gr.inputs.Slider(minimum=1, maximum=10, default=data['sepal_length'].mean(), label='sepal_length')
sepal_width = gr.inputs.Slider(minimum=1, maximum=10, default=data['sepal_width'].mean(), label='sepal_width')
petal_length = gr.inputs.Slider(minimum=1, maximum=10, default=data['petal_length'].mean(), label='petal_length')
petal_width = gr.inputs.Slider(minimum=1, maximum=10, default=data['petal_width'].mean(), label='petal_width')

gui = gr.Interface(fn=predict, inputs=[model,sepal_length,sepal_width,petal_length,petal_width], outputs="label", live=True)

Display the interface

In [ ]:
gui.launch()